In [13]:
import cv2
import numpy as np
import skfuzzy as fuzz
import skfuzzy.control as ctrl
import matplotlib.pyplot as plt

In [14]:
# Carica l'immagine in scala di grigi
image = cv2.imread('./einstein.jpg', cv2.IMREAD_GRAYSCALE)

# Applicare un rilevatore di bordi (es. Sobel o Canny)
edges = cv2.Canny(image, 100, 200)

# Normalizzare l'immagine dei bordi tra 0 e 1
edges_norm = edges / 255.0

# Logica fuzzy per esaltare i bordi
# Definire variabile fuzzy di input (intensità bordi) e output (esaltazione)

bordo = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'bordo')
esaltazione = ctrl.Consequent(np.arange(0, 1.1, 0.1), 'esaltazione')

# Definire le funzioni di appartenenza fuzzy
bordo['debole'] = fuzz.trimf(bordo.universe, [0, 0, 0.5])
bordo['medio'] = fuzz.trimf(bordo.universe, [0, 0.5, 1])
bordo['forte'] = fuzz.trimf(bordo.universe, [0.5, 1, 1])

esaltazione['bassa'] = fuzz.trimf(esaltazione.universe, [0, 0, 0.5])
esaltazione['media'] = fuzz.trimf(esaltazione.universe, [0, 0.5, 1])
esaltazione['alta'] = fuzz.trimf(esaltazione.universe, [0.5, 1, 1])

# Definire le regole fuzzy
rule1 = ctrl.Rule(bordo['debole'], esaltazione['bassa'])
rule2 = ctrl.Rule(bordo['medio'], esaltazione['media'])
rule3 = ctrl.Rule(bordo['forte'], esaltazione['alta'])

# Creare un sistema di controllo fuzzy
esaltazione_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
esaltazione_sim = ctrl.ControlSystemSimulation(esaltazione_ctrl)

# Applicare la logica fuzzy pixel per pixel
output_image = np.zeros_like(edges_norm)

for i in range(edges_norm.shape[0]):
    for j in range(edges_norm.shape[1]):
        esaltazione_sim.input['bordo'] = edges_norm[i, j]
        esaltazione_sim.compute()
        output_image[i, j] = esaltazione_sim.output['esaltazione']

# Convertire l'immagine esaltata in formato 8-bit per visualizzazione
output_image = (output_image * 255).astype(np.uint8)



In [15]:
# Mostrare il risultato
cv2.imshow('Immagine Originale', image)
cv2.imshow('Bordi Esaltati', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Salvare l'immagine del risultato
cv2.imwrite('bordi_esaltati.jpg', output_image)

True